- this file is used for making train data (1) and test data (5) from sensor 1, 2, 5, 9, and 13
- The training data will consist of an accumulation of 70% from each sensor
- the test data will consist of 30% from each sensors

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [34]:
df_1 = pd.read_csv('1/capture20110810.binetflow')
df_2 = pd.read_csv('2/capture20110811.binetflow')
df_5 = pd.read_csv('5/capture20110815-2.binetflow')
df_9 = pd.read_csv('9/capture20110817.binetflow')
df_13 = pd.read_csv('13/capture20110815-3.binetflow')

df = [df_1, df_2, df_5, df_9, df_13]

In [35]:
train_sum = 0
test_sum = 0

for i in df:
    size = len(i)
    train_sum += int(size * 0.7)
    test_sum += int(size * 0.3)
    print(f"{size}\t{int(size * 0.7)}\t{int(size * 0.3)}")

print('train_sum', train_sum)
print('test_sum', test_sum)

2824636	1977245	847390
1808122	1265685	542436
129832	90882	38949
2087508	1461255	626252
1925149	1347604	577544
train_sum 6142671
test_sum 2632571


In [36]:
# simplify the label column
def categorize_label(label):
    label = str(label).lower()
    if 'botnet' in label:
        if 'spam' in label:
            return 'botnet_spam'
        else:
            return 'botnet'
    else:
        return 'normal'

# Apply the function to the 'Label' column
for i in df:
    i['Label'] = i['Label'].apply(categorize_label)

to make the data simpler, (following my paper) this code will remove the unecessary feature such as:
- dTos
- sTos
- ActivityLabel (only in NCC-2)
- BotnetName (only in NCC-2)
- SensorId (only in NCC-2)
- StartTime

In [37]:
for index in range(len(df)):
    df[index] = df[index].drop(
        columns=['dTos', 'sTos', 'ActivityLabel', 'BotnetName', 'SensorId', 'StartTime'], 
        errors='ignore'
    )

#### Splitting

In [38]:
normal_df = []
botnet_df = []
botnet_spam_df = []

for i in df:
    normal_df.append(i[i['Label'] == 'normal'])
    botnet_df.append(i[i['Label'] == 'botnet'])
    botnet_spam_df.append(i[i['Label'] == 'botnet_spam'])

In [39]:
normal_train, normal_test = [], []
botnet_train, botnet_test = [], []
botnet_spam_train, botnet_spam_test = [], []

for a, b, c in zip(normal_df, botnet_df, botnet_spam_df):
    temp_train, temp_test = train_test_split(a, test_size=0.3, random_state=42)
    normal_train.append(temp_train)
    normal_test.append(temp_test)

    temp_train, temp_test = train_test_split(b, test_size=0.3, random_state=42)
    botnet_train.append(temp_train)
    botnet_test.append(temp_test)

    temp_train, temp_test = train_test_split(c, test_size=0.3, random_state=42)
    botnet_spam_train.append(temp_train)
    botnet_spam_test.append(temp_test)

In [40]:
# combining training data into one
temp_train_df = []
for a, b, c in zip(normal_train, botnet_train, botnet_spam_train):
    temp_train_df.append(pd.concat([a, b, c], ignore_index=True))
train_df = pd.concat(temp_train_df, ignore_index=True)

# combining test data for each 
test_df = []
for a, b, c in zip(normal_test, botnet_test, botnet_spam_test):
    test_df.append(pd.concat([a, b, c], ignore_index=True))

# Shuffle the combined training and testing datasets
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
for index in range(len(test_df)):
    test_df[index] = test_df[index].sample(frac=1, random_state=42).reset_index(drop=True)

In [41]:
# Tambahan Algof (25 Oktober 2025 pk 10.34 WIB)
train_df = pd.get_dummies(train_df, columns=['Proto'], prefix='', prefix_sep='')

In [42]:
# Tambahan Algof (25 Oktober 2025 pk 10.34 WIB)
#  Mencari semua possible value untuk fitur `State`
all_states = []

for temp_df in df:
    all_states.extend(temp_df['State'].unique())
    
unique_all_states = list(set(all_states))

del all_states, temp_df

In [43]:
# Tambahan Algof (25 Oktober 2025 pk 10.34 WIB)
# Mapping semua possible value untuk fitur `State` ke dalam numerik 
# dan mengubah fitur `State` pada dataframe original
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(unique_all_states)

train_df['State'] = le.transform(train_df['State'])

del unique_all_states

In [44]:
train_df.to_csv('final_dataset/train.csv', index=False)

In [45]:
filenames = ['1', '2', '3']
for a, b in zip(test_df, filenames):
    # Tambahan Algof (25 Oktober 2025 pk 00.24 WIB)
    a = pd.get_dummies(a, columns=['Proto'], prefix='', prefix_sep='')
    a['State'] = le.transform(a['State'])
    # ---------------------------------------------
    a.to_csv(f'final_dataset/test_{b}.csv', index=False)